In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.core.display import display, HTML, Javascript

# ----- Notebook Theme -----
color_map = ['#00e600', '#e8f3f6', '#d0e8ec', '#a2d1d9', '#73b4c6', '#4597b3', 
                        '#54BAB9', '#13688d', '#11637a', '#115e7a', '#006600']

prompt = color_map[-1]
main_color = color_map[0]
strong_main_color = color_map[1]
custom_colors = [strong_main_color, main_color]

css_file = ''' 

#head {
font-family: monospace
}

div #notebook {
background-color: white;
line-height: 20px;
}

#notebook-container {
%s
margin-top: 2em;
padding-top: 2em;
border-top: 4px solid %s; /* light orange */
-webkit-box-shadow: 0px 0px 8px 2px rgba(224, 212, 226, 0.5); /* pink */
    box-shadow: 0px 0px 8px 2px rgba(224, 212, 226, 0.5); /* pink */
}

div .input {
margin-bottom: 1em;
}

.rendered_html h1, .rendered_html h2, .rendered_html h3, .rendered_html h4, .rendered_html h5, .rendered_html h6 {
color: %s; /* light orange */
font-weight: 600;
}

div.input_area {
border: none;
    background-color: %s; /* rgba(229, 143, 101, 0.1); light orange [exactly #E58F65] */
    border-top: 2px solid %s; /* light orange */
}

div.input_prompt {
color: %s; /* light blue */
}

div.output_prompt {
color: %s; /* strong orange */
}

div.cell.selected:before, div.cell.selected.jupyter-soft-selected:before {
background: %s; /* light orange */
}

div.cell.selected, div.cell.selected.jupyter-soft-selected {
    border-color: %s; /* light orange */
}

.edit_mode div.cell.selected:before {
background: %s; /* light orange */
}

.edit_mode div.cell.selected {
border-color: %s; /* light orange */

}
'''
def to_rgb(h): 
    return tuple(int(h[i:i+2], 16) for i in [0, 2, 4])

main_color_rgba = 'rgba(%s, %s, %s, 0.1)' % (to_rgb(main_color[1:]))
open('notebook.css', 'w').write(css_file % ('width: 95%;', main_color, main_color, main_color_rgba, main_color,  main_color, prompt, main_color, main_color, main_color, main_color))

def nb(): 
    return HTML("<style>" + open("notebook.css", "r").read() + "</style>")
nb()

# Spaceship Titanic

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900; font-size: 30px"> About <span style="color: #2eb82e"><em>Spaceship Titanic</em></span> </h2>

<center><img width=600 height=400 src='https://storage.googleapis.com/kaggle-media/competitions/Spaceship%20Titanic/joel-filipe-QwoNAhbmLLo-unsplash.jpg'></img></center>

<h1 align='center' style="font-family: 'Courier New', monospace; line-height: 5%; font-weight: 600"> Year: <span style="color: #2eb82e">2912</span></h1>
<h1 align='center' style="font-family: 'Courier New', monospace; line-height: 5%;font-weight: 600"> Ship: <span style="color: #2eb82e"><em>Spaceship Titanic</em></span></h1>
<h1 align='center' style="font-family: 'Courier New', monospace; line-height: 5%;font-weight: 600"> Launched: <span style="color: #2eb82e">a month ago</span></h1>
<h1 align='center' style="font-family: 'Courier New', monospace; line-height: 5%;font-weight: 600"> Total Passengers: <span style="color: #2eb82e">about 13,000</span></h1>
<h1 align='center' style="font-family: 'Courier New', monospace; line-height: 5%;font-weight: 600"> Status:  <span style="color: #2eb82e">Lost around Alpha Centauri</span></h1>

<h2 style="font-family: 'Trebuchet MS'; font-size: 20px;"> The ship has travelled to an alternate dimension while it was rounding Alpha Centauri en route to its first destination— the torrid 55 Cancri E by colliding with a space-time anamoly hidden within a dust cloud. Almost half of the passengers are transported to alternate dimension and our job is to predict which passengers are transported.</h2>


# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900; font-size: 30px"> Getting <span style="color: #2eb82e">Libraries</span> </h2>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import plotly.express as px
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PowerTransformer, LabelEncoder, StandardScaler
import optuna
from wordcloud import WordCloud
import warnings 
warnings.filterwarnings('ignore')

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900; font-size: 30px"> Reading <span style="color: #2eb82e">dataset</span> </h2>

In [ ]:
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')

In [ ]:
train.info()

In [ ]:
train.astype('object').describe()

In [ ]:
pd.DataFrame({'Null Count': train.isna().sum()}).T

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900; font-size: 30px"> EDA <span style="color: #00ff00"></span> </h2>

In [ ]:
df = train.copy()
df['Deck'] = df['Cabin'].apply(lambda x:str(x)[:1])
df['Side'] = df['Cabin'].apply(lambda x:str(x)[-1:])
df['Name'].fillna(' noname', inplace=True)
df['LastName'] = df['Name'].apply(lambda x:x.split(" ")[1])

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900;"> Transported</h2>

The target variable i.e. Transported has a uniform distribution of the values (True: passengers were transported & False: they were not). So, I don't have to  worry about upsampling or downsampling the target column. Looking at the below plot, I find out that about 50.4% of passengers were transported during their space voyage while 49.6% of the passengers were lucky.

In [ ]:
fig = px.pie(df, values=df['Transported'].value_counts(), names=['Yes', 'No'], title='Transported (Yes/ No)', template='plotly_dark')
fig.update_traces(pull=[0.05, 0.0], textfont_size=20, marker=dict(colors=['#00e600', '#ff0066'], line=dict(color='white', width=2)))
fig.update_layout(font=dict(
        family="Trebuchet MS, sans-serif",
        size=18,
        color="white"
    ))
fig.show()

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900;"> Home<span style="color: #2eb82e">Planet</span> </h2>

HomePlanet specifies the location from where they started from or their permanent residence. There are three locations from which people departed:
* Earth
* Europa
* Mars

And from the plot below, I come to know that most of the passengers were from Earth (54.2%) while Mars was the residence of least number of passengers (20.7%) out of the other three.

In [ ]:
fig = px.pie(df, values=df['HomePlanet'].value_counts(), names=['Earth', 'Europa', 'Mars'], template='plotly_dark', title='HomePlanet')
fig.update_traces(pull=[0.03, 0.0, 0.05], textfont_size=20, marker=dict(colors=['#00e600', '#ff0066', '#1ad1ff'], line=dict(color='white', width=2)))
fig.update_layout(font=dict(
        family="Trebuchet MS, sans-serif",
        size=18,
        color="white"
    ))
fig.show()

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900;"> Destination</h2>

Destination is the location where the passengers were going to. They are also three locations:
* TRAPPIST-1e
* 55 Cancri e
* PSO J318.5-22

TRAPPIST-1e was the most favourite destination of passengers.


In [ ]:
fig = px.pie(df, values=df['Destination'].value_counts(), names=['TRAPPIST-1e', '55 Cancri e', 'PSO J318.5-22'], template='plotly_dark', title='Destination')
fig.update_traces(pull=[0.0, 0.04, 0.2], textfont_size=20, marker=dict(colors=['#00e600', '#ff0066', '#1ad1ff'], line=dict(color='white', width=2)))
fig.update_layout(font=dict(
        family="Trebuchet MS, sans-serif",
        size=18,
        color="white"
    ))
fig.show()

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900;"> Cryo<span style="color: #2eb82e">Sleep</span> </h2>

Looking at the relationship between CryoSleep and the Transported variable, I think that most of the people who were transported were in CryoSleep and those who didn't were awake. Overall, the number of people who were not in CryoSleep are more than those who were. So, this means that CryoSleep is one of the most important variable that I require for predicting the target variable: Transported.

In [ ]:
plt.figure(figsize=[15, 8])
sns.set_style('dark')
sns.set_palette('bright')
plt.subplot(1, 2, 1)
sns.countplot(data=df, x='CryoSleep')
plt.subplot(1, 2, 2)
sns.countplot(data=df, x='CryoSleep', hue='Transported')
plt.tight_layout()

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900;"> VIP</h2>

From the plot below, I come to know that most of the passengers haven't paid for special VIP services. I'll keep this variable for final correlation checking.

In [ ]:
plt.figure(figsize=[15, 8])
plt.subplot(1, 3, 1)
sns.countplot(data=df, x='VIP')
plt.subplot(1, 3, 2)
sns.countplot(data=df, x='VIP', hue='CryoSleep')
plt.subplot(1, 3, 3)
sns.countplot(data=df, x='VIP', hue='Transported')
plt.tight_layout()

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900;"> Age</h2>

Seeing the plot, I can observe that the plot of Age vs Transported is not that much distinguishable. Hence, I can conclude that the target variable is not effectively charactersized by Age. But still it gives me some information. I get to know that most of the passengers were in their mid-twenties or thirties. Maybe this justifies more people not taking special VIP services just like the Pclass in Titanic.

In [ ]:
plt.figure(figsize=[15, 8])
plt.subplot(1, 2, 1)
sns.kdeplot(data=df, x='Age')
plt.subplot(1, 2, 2)
sns.kdeplot(data=df, x='Age', hue='Transported')
plt.tight_layout()

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900;"> Cabin: <span style="color: #2eb82e">Deck</span> & <span style="color: #2eb82e">Side</span> </h2>

Looking at the Cabin column as a whole doesn't helps me but decomposing it into Deck and Side can play an important role in predicting whether the passengers were transported or not. 
So, from the plot below, I see that:
* Most of the passengers belong to Deck F & G; and
* the side P (Port) and S (Starboard).

In [ ]:
plt.figure(figsize=[15, 8])
plt.subplot(1, 2, 1)
sns.countplot(data=df, x='Deck')
plt.subplot(1, 2, 2)
sns.countplot(data=df, x='Side')
plt.tight_layout()

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900;"> Other <span style="color: #2eb82e">numeric variables</span> </h2>

Let's look at other numeric variables.

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(40, 10))
sns.distplot(df['RoomService'], ax=ax[0])
sns.distplot(df['FoodCourt'], ax=ax[1])
sns.distplot(df['ShoppingMall'], ax=ax[2])
sns.distplot(df['Spa'], ax=ax[3])
sns.distplot(df['VRDeck'], ax=ax[4])
plt.show()

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900;"> WordCloud: <span style="color: #2eb82e">Last Name</span> </h2>

I think that Name feature can be used to create a new feature to determine families or solo trips. This can also be a good variable for predicting the target column. I'll use word cloud to visualize this. (I've replaced null values with "noname")

In [ ]:
text = " ".join(i for i in df['LastName'])
word_cloud = WordCloud(width=1600, height=800, collocations = False, background_color = 'white').generate(text)
plt.figure(figsize=[14, 8])
plt.imshow(word_cloud)
plt.axis("off")
plt.show()

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900; font-size:30px"> Pre-<span style="color: #2eb82e">processing</span> </h2>

First, to tackle null values in numeric columns, I'll impute the missing numeric values using SimpleImputer. 

In [ ]:
imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService"]
imputer = SimpleImputer()
imputer.fit(train[imputer_cols])
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])

In [ ]:
pd.DataFrame({'Null Count': train.isna().sum()}).T

Second, to tackle null values of categorical variables, I'll replace the null values with the most frequent occuring value of that particular column.

In [ ]:
train['HomePlanet'].fillna('Earth', inplace=True)
train['CryoSleep'].fillna('False', inplace=True)
train['Cabin'].fillna('None', inplace=True)
train['Destination'].fillna('TRAPPIST-1e', inplace=True)
train['VIP'].fillna('False', inplace=True)
# train['RoomService'].fillna(0, inplace=True)
# train['FoodCourt'].fillna(0, inplace=True)
# train['ShoppingMall'].fillna(0, inplace=True)
# train['Spa'].fillna(0, inplace=True)
# train['VRDeck'].fillna(0, inplace=True)
# train['Age'].fillna(27.0, inplace=True)

test['HomePlanet'].fillna('Earth', inplace=True)
test['CryoSleep'].fillna('False', inplace=True)
test['Cabin'].fillna('G/109/P', inplace=True)
test['Destination'].fillna('TRAPPIST-1e', inplace=True)
test['VIP'].fillna('False', inplace=True)
# test['RoomService'].fillna(0, inplace=True)
# test['FoodCourt'].fillna(0, inplace=True)
# test['ShoppingMall'].fillna(0, inplace=True)
# test['Spa'].fillna(0, inplace=True)
# test['VRDeck'].fillna(0, inplace=True)
# test['Age'].fillna(27.0, inplace=True)

In [ ]:
pd.DataFrame({'Null Count': train.isna().sum()}).T

Third, I'll replace the empty value in Name column with "no name".

In [ ]:
train['Name'].fillna('no name', inplace=True)
test['Name'].fillna('no name', inplace=True)

In [ ]:
pd.DataFrame({'Null Count': train.isna().sum()}).T

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900; font-size:30px"> Feature <span style="color: #2eb82e">Engineering</span> </h2>

I'll create the following columns:
* Deck: will store the cabin deck number
* Side: will store the cabin side
* PGroup: will store the Passenger Group no.
* PNumber: will store the Passenger no. in that group
* LastName: will store the last name of the passenger
* AllServices: will store the combined value of all the services

In [ ]:
train['Deck'] = train['Cabin'].apply(lambda x:str(x)[:1])
train['Side'] = train['Cabin'].apply(lambda x:str(x)[-1:])
test['Deck'] = train['Cabin'].apply(lambda x:str(x)[:1])
test['Side'] = train['Cabin'].apply(lambda x:str(x)[-1:])

train['PGroup'] = train['PassengerId'].apply(lambda x: x.split('_')[0])
train['PNumber'] = train['PassengerId'].apply(lambda x: x.split('_')[1])
test['PGroup'] = test['PassengerId'].apply(lambda x: x.split('_')[0])
test['PNumber'] = test['PassengerId'].apply(lambda x: x.split('_')[1])

train['LastName'] = train['Name'].apply(lambda x:x.split(" ")[1])
test['LastName'] = train['Name'].apply(lambda x:x.split(" ")[1])

train['AllServices'] = train['RoomService'] + train['FoodCourt'] + train['ShoppingMall'] + train['Spa'] + train['VRDeck']
test['AllServices'] = test['RoomService'] + test['FoodCourt'] + test['ShoppingMall'] + test['Spa'] + test['VRDeck']

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900; font-size:30px"> Cor<span style="color: #2eb82e">relation</span> </h2>

In [ ]:
plt.figure(figsize=[18, 8])
sns.heatmap(data=train.corr(), annot=True, linewidths=1, linecolor='black', cmap='magma')

In [ ]:
label_cols = ["HomePlanet", "CryoSleep", "Destination" ,"VIP", "Deck", "Side", "LastName", "PGroup", "PNumber"]

for i in label_cols:
    train[i] = train[i].astype(str)
    test[i] = test[i].astype(str)
    train[i] = LabelEncoder().fit_transform(train[i])
    test[i] = LabelEncoder().fit_transform(test[i])

# <h2 style="font-family: 'Trebuchet MS'; font-weight: 900; font-size:30px"> Model <span style="color: #2eb82e">Training</span> + <span style="color: #2eb82e">Optimization</span> </span> </h2>

Now, I have done the major part, I can move on to model training. Let's look at the modified dataset.

In [ ]:
train

In [ ]:
test

Now, I don't need Name, PassengerId and Cabin columns.

In [ ]:
cols_to_drop = ['Name', 'PassengerId', 'Cabin']

In [ ]:
train.drop(cols_to_drop, axis=1, inplace=True)
test.drop(['Name', 'Cabin'], axis=1, inplace=True)
ids = test['PassengerId']

test.drop('PassengerId', axis=1, inplace=True)

I'll be using LightGBM for model creation and prediction and will optimize it using Optuna.

In [ ]:
def lgbtuner(trial):
    cols = [x for x in train.columns if (x != 'Name') and (x != 'PassengerId') and (x != 'Cabin') and (x != 'Transported')]
    target = 'Transported'
    
    strat_split = StratifiedShuffleSplit(n_splits=3, test_size=0.4)
    train_idx, test_idx = next(strat_split.split(train[cols], train[target]))
    
    x_train = train.loc[train_idx, cols]
    y_train = train.loc[train_idx, target]
    
    x_test = train.loc[test_idx, cols]
    y_test = train.loc[test_idx, target]
    
    label_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService", "AllServices"]

    for i in label_cols:
        x_train[i] = StandardScaler().fit_transform(np.array(x_train[i]).reshape(-1, 1))
        x_test[i] = StandardScaler().fit_transform(np.array(x_test[i]).reshape(-1, 1))
    
    params = {
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100)
    }
    
    lgbm = lgb.LGBMClassifier(**params)
    lgbm.fit(x_train, y_train)
    
    pred = lgbm.predict(x_test)
    score = accuracy_score(y_test, pred)
    
    return score

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(lgbtuner, n_trials = 100)

In [ ]:
study.best_params

In [ ]:
# lgbp = {'lambda_l1': 0.03677669268217501,
#  'lambda_l2': 5.2925099897210586e-05,
#  'num_leaves': 183,
#  'feature_fraction': 0.9774637656586244,
#  'bagging_fraction': 0.9218866217855697,
#  'bagging_freq': 3,
#  'min_child_samples': 100}

lgbp = {'lambda_l1': 0.0009453028824990722,
 'lambda_l2': 1.776610136408957,
 'num_leaves': 19,
 'feature_fraction': 0.6355091532859224,
 'bagging_fraction': 0.6948362254424815,
 'bagging_freq': 2,
 'min_child_samples': 36}

In [ ]:
cols = [x for x in train.columns if (x != 'Name') and (x != 'PassengerId') and (x != 'Cabin') and (x != 'Transported')]
target = 'Transported'

strat_split = StratifiedShuffleSplit(n_splits=3, test_size=0.4)
train_idx, test_idx = next(strat_split.split(train[cols], train[target]))

x_train = train.loc[train_idx, cols]
y_train = train.loc[train_idx, target]

x_test = train.loc[test_idx, cols]
y_test = train.loc[test_idx, target]

In [ ]:
lgbm = lgb.LGBMClassifier(**lgbp)
lgbm.fit(x_train, y_train)
pred = lgbm.predict(test)
pred

In [ ]:
subs = pd.DataFrame({'PassengerId': ids, 'Transported': pred})
subs

In [ ]:
subs.to_csv('subs.csv', index=False)